In [ ]:
import easyocr
import os
import torch
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import random
import glob
from tqdm import tqdm
import numpy as np
import time
from mmocr_eval import eval_ocr_metric
import re
import cv2 as cv

In [ ]:
# !pip install onnx
# !pip install onnxruntime-gpu

In [ ]:
torch.cuda.is_available()

In [ ]:
model = 'crnn_30k_lr_25e-2' #folder name of model
iteration = 'crnn_30k_lr_25e-2' # model .pth file name

model_path = os.path.join('trainer', 'saved_models',model)
reader = easyocr.Reader(lang_list = ['en','th'],
                    model_storage_directory = model_path,
                    user_network_directory = os.path.join('my_model','user_network'),
                    recog_network  = iteration,
                    config_path = 'crnn', #configuration file name use to get character_list only  (so able to use for every model)
                    gpu = True)
# reader = easyocr.Reader(lang_list = ['en','th'],gpu = True )

In [ ]:
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
random_file_path = random.sample(filepaths,1)[0]
img = plt.imread(random_file_path)
result = reader.recognize(random_file_path)
result

In [ ]:
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# # filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))[:1000]
# filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
# label_path = 'trainer/all_data/testing/testing/label.txt'
# with open(label_path, encoding='utf8') as file:
#     label = np.loadtxt(file,dtype=str)
# start = time.time()
# # filepaths = filepaths[:10]
# pred = []
# gt = [] # ground truth
# name_img = []
# for i in tqdm(range(len(filepaths))):
#     label_img = label[i][1]
#     name_img.append(label[i][0])
#     result = reader.recognize(filepaths[i],detail=0)
#     try:
#         pred.append(result)
#     except:
#         pred.append('')
#     gt.append(label_img)
# end = time.time()
# print(f'time: {end - start}')

In [ ]:
# tmp = name_img.copy()
# output, false_list = eval_ocr_metric(pred, gt, tmp)
# # output, false_list = eval_ocr_metric(pred, gt, name_img)
# for k,v in output.items():
#     print(f'{k}: {v}')